In [36]:
import pandas as pd
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, log, exp, bioDraws, bioMultSum, MonteCarlo, bioNormalCdf, Elem, RandomVariable, Integrate, PanelLikelihoodTrajectory, Derive
import biogeme.database as db
import numpy as np
import biogeme.distributions as dist
import biogeme.exceptions as excep
import biogeme.results as res
import biogeme.loglikelihood as ll
from biogeme.results import bioResults

In [37]:
df = pd.read_csv('biogeme_data.csv')

In [38]:
df['lowtreat'] = df['treatment'].replace({0: 1, 1: 0, 2: 0})
df['hightreat'] = df['treatment'].replace({0: 0, 1: 0, 2: 1})

In [39]:
df['notcar'] = df['vehclass'].replace({1:0, 2:1, 3:1, 4:1, 5:1})
df['CR_less0'] = np.where(df['cr_fs'] < 0, 1, 0)
df['age_51p'] = np.where(df['age'] >= 50, 1, 0)
df['female'] = np.where(df['gender'] == 2, 1, 0)
df['children'] = np.where(df['children'] > 0, 1, 0)
df['worker'] = np.where((df['employment'] == 2) | (df['employment'] == 3), 1, 0)
df['white'] = np.where(df['race'] == 'White', 1, 0)
df['chargavail_fraction'] = df['ev_chargavail'] / 100

In [41]:
df = df.drop('race', axis=1)

In [42]:
database = db.Database('df', df)
# mixed logit model
# database.panel('resp_index')
globals().update(database.variables)

In [9]:
# MODELNAME = 'PR_StructuralModel'
# struct_results = res.bioResults(pickleFile=f'{MODELNAME}.pickle')
# reliab_betas = struct_results.getBetaValues()

FileNotFoundError: File PR_StructuralModel.pickle not found

In [43]:
PR_intercept = Beta('PR_Intercept', 0.062, None, None, 0)
PR_lowtreat = Beta('PR_lowtreat', -0.758, None, None, 0)
PR_hightreat = Beta('PR_hightreat', 0.573, None, None, 0)

# PR_intercept = Beta('PR_Intercept', 0, None, None, 0)
# PR_lowtreat = Beta('PR_lowtreat', -0, None, None, 0)
# PR_hightreat = Beta('PR_hightreat', 0, None, None, 0)

In [44]:
sigma_s = Beta('sigma_s', 0.1, None, None, 0)
ec = sigma_s * bioDraws('ec', 'NORMAL')

omega = RandomVariable('omega')
density = dist.normalpdf(omega)

In [45]:
PR = (
    PR_intercept
    + PR_hightreat*hightreat
    + PR_lowtreat*lowtreat
    + ec
    # + sigma_s * omega
)

In [46]:
# INTER_Reliab01 = Beta('INTER_Reliab01', 0, None, None, 0)
INTER_Reliab02 = Beta('INTER_Reliab02', 0, None, None, 1)
INTER_Reliab03 = Beta('INTER_Reliab03', -0.040, None, None, 0)
INTER_Reliab04 = Beta('INTER_Reliab04', -0.130, None, None, 0)
INTER_Reliab05 = Beta('INTER_Reliab05', -0.356, None, None, 0)

In [47]:
# B_Reliab01_F1 = Beta('B_Reliab01_F1', 1, None, None, 0)
B_Reliab02_F1 = Beta('B_Reliab02_F1', 1, None, None, 1)
B_Reliab03_F1 = Beta('B_Reliab03_F1', 1.010, None, None, 0)
B_Reliab04_F1 = Beta('B_Reliab04_F1',0.959, None, None, 0)
B_Reliab05_F1 = Beta('B_Reliab05_F1', 1.030, None, None, 0)

In [48]:
# MODEL_Reliab01 = INTER_Reliab01 + B_Reliab01_F1 * PR
MODEL_Reliab02 = INTER_Reliab02 + B_Reliab02_F1 * PR
MODEL_Reliab03 = INTER_Reliab03 + B_Reliab03_F1 * PR
MODEL_Reliab04 = INTER_Reliab04 + B_Reliab04_F1 * PR
MODEL_Reliab05 = INTER_Reliab05 + B_Reliab05_F1 * PR

In [49]:
SIGMA_STAR_Reliab01 = Beta('SIGMA_STAR_Reliab01', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab02 = Beta('SIGMA_STAR_Reliab02', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab03 = Beta('SIGMA_STAR_Reliab03', 0.867, 1.0e-5, None, 0)
SIGMA_STAR_Reliab04 = Beta('SIGMA_STAR_Reliab04', 0.744, 1.0e-5, None, 0)
SIGMA_STAR_Reliab05 = Beta('SIGMA_STAR_Reliab05', 1.100, 1.0e-5, None, 0)

In [50]:
delta_1 = Beta('delta_1',1.350,0,10,0)
delta_2 = Beta('delta_2',1.230,0,10,0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

In [51]:
Reliab02_tau_1 = (tau_1-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_2 = (tau_2-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_3 = (tau_3-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_4 = (tau_4-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_5 = (tau_5-MODEL_Reliab02) / SIGMA_STAR_Reliab02

Reliab03_tau_1 = (tau_1-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_2 = (tau_2-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_3 = (tau_3-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_4 = (tau_4-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_5 = (tau_5-MODEL_Reliab03) / SIGMA_STAR_Reliab03

Reliab04_tau_1 = (tau_1-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_2 = (tau_2-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_3 = (tau_3-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_4 = (tau_4-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_5 = (tau_5-MODEL_Reliab04) / SIGMA_STAR_Reliab04

Reliab05_tau_1 = (tau_1-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_2 = (tau_2-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_3 = (tau_3-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_4 = (tau_4-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_5 = (tau_5-MODEL_Reliab05) / SIGMA_STAR_Reliab05

IndReliab02 = {
    1: bioNormalCdf(Reliab02_tau_1),
    2: bioNormalCdf(Reliab02_tau_2)-bioNormalCdf(Reliab02_tau_1),
    3: bioNormalCdf(Reliab02_tau_3)-bioNormalCdf(Reliab02_tau_2),
    4: bioNormalCdf(Reliab02_tau_4)-bioNormalCdf(Reliab02_tau_3),
    5: bioNormalCdf(Reliab02_tau_5)-bioNormalCdf(Reliab02_tau_4),
    6: 1-bioNormalCdf(Reliab02_tau_5),
    9999: 1.0
}

P_Reliab02 = Elem(IndReliab02, reliab_2)

IndReliab03 = {
    1: bioNormalCdf(Reliab03_tau_1),
    2: bioNormalCdf(Reliab03_tau_2)-bioNormalCdf(Reliab03_tau_1),
    3: bioNormalCdf(Reliab03_tau_3)-bioNormalCdf(Reliab03_tau_2),
    4: bioNormalCdf(Reliab03_tau_4)-bioNormalCdf(Reliab03_tau_3),
    5: bioNormalCdf(Reliab03_tau_5)-bioNormalCdf(Reliab03_tau_4),
    6: 1-bioNormalCdf(Reliab03_tau_5),
    9999: 1.0
}

P_Reliab03 = Elem(IndReliab03, reliab_3)

IndReliab04 = {
    1: bioNormalCdf(Reliab04_tau_1),
    2: bioNormalCdf(Reliab04_tau_2)-bioNormalCdf(Reliab04_tau_1),
    3: bioNormalCdf(Reliab04_tau_3)-bioNormalCdf(Reliab04_tau_2),
    4: bioNormalCdf(Reliab04_tau_4)-bioNormalCdf(Reliab04_tau_3),
    5: bioNormalCdf(Reliab04_tau_5)-bioNormalCdf(Reliab04_tau_4),
    6: 1-bioNormalCdf(Reliab04_tau_5),
    9999: 1.0
}

P_Reliab04 = Elem(IndReliab04, reliab_4)

IndReliab05 = {
    1: bioNormalCdf(Reliab05_tau_1),
    2: bioNormalCdf(Reliab05_tau_2)-bioNormalCdf(Reliab05_tau_1),
    3: bioNormalCdf(Reliab05_tau_3)-bioNormalCdf(Reliab05_tau_2),
    4: bioNormalCdf(Reliab05_tau_4)-bioNormalCdf(Reliab05_tau_3),
    5: bioNormalCdf(Reliab05_tau_5)-bioNormalCdf(Reliab05_tau_4),
    6: 1-bioNormalCdf(Reliab05_tau_5),
    9999: 1.0
}

P_Reliab05 = Elem(IndReliab05, reliab_5)

In [52]:
# ASC_EV_1_HOMECHARGE = Beta('ASC_EV_1_HOMECHARGE', reliab_betas['ASC_EV_1'], None, None, 0)
# ASC_EV_1_NOHOMECHARGE = Beta('ASC_EV_1_NOHOMECHARGE', reliab_betas['ASC_EV_1'], None, None, 0)
ASC_EV_1 = Beta('ASC_EV_1', -3.031209, None, None, 0)
ASC_EV_1_NoHome = Beta('ASC_EV_1_NoHome', -0.276, None, None, 0)
B_PRICE_EV_1 = Beta('B_PRICE_EV_1', -2.640, None, None, 0)
B_PRICE_CONV_1 = Beta('B_PRICE_CONV_1', -2.380, None, None, 0)
B_OPCOST_EV_1 = Beta('B_OPCOST_EV_1', -0.080, None, None, 0)
B_OPCOST_CONV_1 = Beta('B_OPCOST_CONV_1', -0.071, None, None, 0)
B_RANGE_EV_1 = Beta('B_RANGE_EV_1', 0.17, None, None, 0)
B_RANGE_CONV_1 = Beta('B_RANGE_CONV_1', 0.15, None, None, 0)
B_CHARGAVAIL_EV_1_HOME = Beta('B_CHARGAVAIL_EV_1_HOME', 0.443, None, None, 0)
B_CHARGAVAIL_EV_1_NOHOME = Beta('B_CHARGAVAIL_EV_1_NOHOME', 0.0225, None, None, 0)
B_CHARGAVAIL_EV_1 = Beta('B_CHARGAVAIL_EV_1', 0.2, None, None, 0)
B_PR_1_NOHOME = Beta('B_PR_1_NOHOME', 0.307470, None, None, 0)
B_PR_1 = Beta('B_PR_1', 0.734057, None, None, 0)

ASC_EV_1_S = Beta('ASC_EV_1_S', 0.832111, None, None, 0)
ASC_EV_1_RND = ASC_EV_1 + ASC_EV_1_S * bioDraws('ASC_EV_1_RND', 'NORMAL')

# B_PR_S = Beta('B_PR_S', 0.1, None, None, 0)
# B_PR_1_RND = B_PR_1 + B_PR_S * bioDraws('B_PR_1_RND', 'NORMAL_MLHS')




In [53]:
V_EV_1 = (
    ASC_EV_1 #_RND
    + ASC_EV_1_NoHome*(1-homecharge)
    + B_PRICE_EV_1* (ev_price/budget)
    + B_OPCOST_EV_1*ev_opcost
    + B_RANGE_EV_1*(ev_range/100)
    + B_CHARGAVAIL_EV_1*chargavail_fraction
    + B_CHARGAVAIL_EV_1_NOHOME*chargavail_fraction*(1-homecharge)
    + B_PR_1*PR
    + B_PR_1_NOHOME*PR*(1-homecharge)
)

V_CONV_1 = (
    B_PRICE_CONV_1* (icev_price/budget)
    + B_OPCOST_CONV_1*icev_opcost
    + B_RANGE_CONV_1*(icev_range/100)
)


In [54]:
V1 = {1: V_EV_1, 2: V_CONV_1}


In [55]:

av = {1: ev_av, 2: icev_av}

In [23]:
prob1 = models.logit(V1, av, choice)
condlike = prob1 * P_Reliab03 * P_Reliab04 * P_Reliab05 * P_Reliab02
#condlikeindiv = PanelLikelihoodTrajectory(condlike)
# loglike = log(MonteCarlo(condlikeindiv))s

loglike = log(MonteCarlo(condlike))
# loglike = log(Integrate(condlikeindiv * density, 'omega'))


NameError: name 'V1' is not defined

In [26]:
the_biogeme = bio.BIOGEME(database, loglike,numberOfDraws = 1000, numberOfThreads = 100, bootstrap = 500)
the_biogeme.modelName = 'PR_ICLV_Mixed_MonteCarlo'




The use of argument numberOfThreads in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_threads in section MultiThreading of the .toml parameter file. The default file name is biogeme.toml
The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarlo of the .toml parameter file. The default file name is biogeme.toml


In [27]:
model_results = the_biogeme.estimate(run_bootstrap=False)
pandasResults = model_results.getEstimatedParameters()
print(pandasResults)

                             Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_EV_1                 -1.892825      0.253940    -7.453833  9.059420e-14
ASC_EV_1_NoHome          -0.278185      0.093984    -2.959903  3.077360e-03
B_CHARGAVAIL_EV_1         1.351769      0.074219    18.213219  0.000000e+00
B_CHARGAVAIL_EV_1_NOHOME  0.175561      0.134884     1.301566  1.930648e-01
B_OPCOST_CONV_1          -0.070984      0.005704   -12.443567  0.000000e+00
B_OPCOST_EV_1            -0.079824      0.005717   -13.961407  0.000000e+00
B_PRICE_CONV_1           -2.371916      0.180227   -13.160689  0.000000e+00
B_PRICE_EV_1             -2.623049      0.180417   -14.538788  0.000000e+00
B_PR_1                    0.431738      0.017636    24.480892  0.000000e+00
B_PR_1_NOHOME             0.024996      0.025420     0.983344  3.254381e-01
B_RANGE_CONV_1            0.158459      0.024523     6.461568  1.036236e-10
B_RANGE_EV_1              0.230119      0.024368     9.443430  0.000000e+00
B_Reliab03_F

Market Shares from Original Data

In [28]:

prob_EV_indiv = MonteCarlo(models.logit(V1, av, 1))
prob_ICEV_indiv = MonteCarlo(models.logit(V1, av, 2))

simulate = {
  'prob EV': prob_EV_indiv,
  'prob ICEV': prob_ICEV_indiv
}

biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = 'Reliab_Latent_Simple_Simulation'

betas = biogeme.freeBetaNames

results = res.bioResults(pickleFile = 'PR_ICLV_Mixed_MonteCarlo.pickle')
betaValues = results.getBetaValues()

simulatedValues = biogeme.simulate(betaValues)
simulatedValues.mean()

NameError: name 'V1' is not defined

In [56]:
V_EV_1 = (
    ASC_EV_1 #_RND
    + ASC_EV_1_NoHome*(1-homecharge)
    + B_PRICE_EV_1* (ev_price/budget)
    + B_OPCOST_EV_1*ev_opcost
    + B_RANGE_EV_1*(ev_range/100)
    + B_CHARGAVAIL_EV_1*chargavail_fraction
    + B_CHARGAVAIL_EV_1_NOHOME*chargavail_fraction*(1-homecharge)
    + B_PR_1*(PR-0.960)
    + B_PR_1_NOHOME*(PR-0.960)*(1-homecharge)
)

V_CONV_1 = (
    B_PRICE_CONV_1* (icev_price/budget)
    + B_OPCOST_CONV_1*icev_opcost
    + B_RANGE_CONV_1*(icev_range/100)
)


In [57]:
V1 = {1: V_EV_1, 2: V_CONV_1}

In [58]:

prob_EV_indiv = MonteCarlo(models.logit(V1, av, 1))
prob_ICEV_indiv = MonteCarlo(models.logit(V1, av, 2))

simulate = {
  'prob EV': prob_EV_indiv,
  'prob ICEV': prob_ICEV_indiv
}

biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = 'Reduced_PR'

betas = biogeme.freeBetaNames

results = res.bioResults(pickleFile = 'PR_ICLV_Mixed_MonteCarlo.pickle')
betaValues = results.getBetaValues()

simulatedValues = biogeme.simulate(betaValues)
simulatedValues.mean()

Parameter B_Reliab03_F1 not present in the model.
Parameter B_Reliab04_F1 not present in the model.
Parameter B_Reliab05_F1 not present in the model.
Parameter INTER_Reliab03 not present in the model.
Parameter INTER_Reliab04 not present in the model.
Parameter INTER_Reliab05 not present in the model.
Parameter SIGMA_STAR_Reliab03 not present in the model.
Parameter SIGMA_STAR_Reliab04 not present in the model.
Parameter SIGMA_STAR_Reliab05 not present in the model.
Parameter delta_1 not present in the model.
Parameter delta_2 not present in the model.


prob EV      0.130965
prob ICEV    0.869035
dtype: float64

Market Shares with Weighted Data

In [31]:
df3 = pd.read_csv('biogeme_data_weighted.csv')
df3 = df3.drop('race', axis=1)
df3['chargavail_fraction'] = df3['ev_chargavail'] / 100
df3['lowtreat'] = df3['treatment'].replace({0: 1, 1: 0, 2: 0})
df3['hightreat'] = df3['treatment'].replace({0: 0, 1: 0, 2: 1})
df3['evprice_adj'] = (df3['ev_price'] / df3['budget'])*1.0 #260
db3 = db.Database('df3', df3)
globals().update(db3.variables)

results = res.bioResults(pickleFile = 'PR_ICLV_Mixed_MonteCarlo.pickle')
betaValues = results.getBetaValues()

In [32]:
V_EV_SIM_2 = (
     ASC_EV_1
    + ASC_EV_1_NoHome*(1-homecharge)
    + B_PRICE_EV_1* (ev_price/budget)
    + B_OPCOST_EV_1*ev_opcost
    + B_RANGE_EV_1*(ev_range/100)
    + B_CHARGAVAIL_EV_1*chargavail_fraction
    + B_CHARGAVAIL_EV_1_NOHOME*chargavail_fraction*(1-homecharge)
    + B_PR_1*(PR)
    + B_PR_1_NOHOME*(PR)*(1-homecharge)
)

V_CONV_SIM_2 = (
    B_PRICE_CONV_1* (icev_price/budget)
    + B_OPCOST_CONV_1*icev_opcost
    + B_RANGE_CONV_1*icev_range
)

VSIM_2 = {1: V_EV_SIM_2, 2: V_CONV_SIM_2}

prob_EV_sim = MonteCarlo(models.logit(VSIM_2, av, 1))
prob_ICEV_sim = MonteCarlo(models.logit(VSIM_2, av, 2))


simulate3 = {
    'prob EV': prob_EV_sim,
    'prob ICEV': prob_ICEV_sim
}

biosim = bio.BIOGEME(db3, simulate3)
biosim.modelName = 'NewSim'

simResults = biosim.simulate(results.getBetaValues())
simResults.mean()

Parameter B_Reliab03_F1 not present in the model.
Parameter B_Reliab04_F1 not present in the model.
Parameter B_Reliab05_F1 not present in the model.
Parameter INTER_Reliab03 not present in the model.
Parameter INTER_Reliab04 not present in the model.
Parameter INTER_Reliab05 not present in the model.
Parameter SIGMA_STAR_Reliab03 not present in the model.
Parameter SIGMA_STAR_Reliab04 not present in the model.
Parameter SIGMA_STAR_Reliab05 not present in the model.
Parameter delta_1 not present in the model.
Parameter delta_2 not present in the model.


prob EV      3.587029e-12
prob ICEV    1.000000e+00
dtype: float64